In [1]:
import time
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import emoji
import nltk
import warnings
warnings.filterwarnings("ignore")

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [2]:
meals = pd.read_csv('data_hyderabad/data_preprocessed_classification.csv')

In [3]:
meals.head()

,Restaurant,Review,Review_Preprocessed,Cuisines,meals
0,Beyond Flavours,"The ambience was good, food was quite good . h...","[('ambience', 'NN'), ('good', 'JJ'), ('food', ...","Chinese, Continental, Kebab, European, South I...",[]
1,Beyond Flavours,Ambience is too good for a pleasant evening. S...,"[('ambience', 'NN'), ('good', 'JJ'), ('pleasan...","Chinese, Continental, Kebab, European, South I...",[]
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...,"[('must', 'MD'), ('try', 'VB'), ('great', 'JJ'...","Chinese, Continental, Kebab, European, South I...",['Penne Alfredo Pasta']
3,Beyond Flavours,Soumen das and Arun was a great guy. Only beca...,"[('soumen', 'NNP'), ('das', 'NNS'), ('arun', '...","Chinese, Continental, Kebab, European, South I...",[]
4,Beyond Flavours,Food is good. we ordered Kodi drumsticks and b...,"[('food', 'NN'), ('good', 'JJ'), ('ordered', '...","Chinese, Continental, Kebab, European, South I...","['Kodi drumsticks', 'basket mutton biryani']"


In [4]:
meals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9955 entries, 0 to 9954
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Restaurant           9955 non-null   object
 1   Review               9943 non-null   object
 2   Review_Preprocessed  9955 non-null   object
 3   Cuisines             9955 non-null   object
 4   meals                9955 non-null   object
dtypes: object(5)
memory usage: 389.0+ KB


In [5]:
meals['meals'] = meals['meals'].apply(lambda x: eval(x) if isinstance(x, str) else x)
meals_df = meals.explode('meals')

In [6]:
meals_df = meals_df.dropna(subset=['meals'])

In [7]:
meal_counts = meals_df['meals'].value_counts()

In [8]:
print(meal_counts)

chicken                   658
biryani                   459
rice                      309
pizza                     266
veg                       223
                         ... 
Red Rooster pizza           1
 soups                      1
ato                         1
fiery chicken               1
pepper chicken starter      1
Name: meals, Length: 8990, dtype: int64


In [9]:
meals_df['meals'] = meals_df['meals'].apply(lambda x: x.lower())

In [10]:
meal_counts_lower = meals_df['meals'].value_counts()
print(meal_counts_lower)

chicken                   750
biryani                   627
rice                      359
pizza                     318
veg                       257
                         ... 
candies                     1
fruits cookies              1
keshar pista                1
paninis                     1
pepper chicken starter      1
Name: meals, Length: 7244, dtype: int64


In [11]:
meals_df = meals_df[meals_df['meals'].str.len() > 2]

In [12]:
meals_df.shape

(25449, 5)

In [13]:
meals_df = meals_df.drop_duplicates(subset=['meals'])

In [14]:
meals_df

,Restaurant,Review,Review_Preprocessed,Cuisines,meals
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...,"[('must', 'MD'), ('try', 'VB'), ('great', 'JJ'...","Chinese, Continental, Kebab, European, South I...",penne alfredo pasta
4,Beyond Flavours,Food is good. we ordered Kodi drumsticks and b...,"[('food', 'NN'), ('good', 'JJ'), ('ordered', '...","Chinese, Continental, Kebab, European, South I...",kodi drumsticks
4,Beyond Flavours,Food is good. we ordered Kodi drumsticks and b...,"[('food', 'NN'), ('good', 'JJ'), ('ordered', '...","Chinese, Continental, Kebab, European, South I...",basket mutton biryani
7,Beyond Flavours,Well after reading so many reviews finally vis...,"[('well', 'RB'), ('reading', 'VBG'), ('many', ...","Chinese, Continental, Kebab, European, South I...",corn
7,Beyond Flavours,Well after reading so many reviews finally vis...,"[('well', 'RB'), ('reading', 'VBG'), ('many', ...","Chinese, Continental, Kebab, European, South I...",tawa fish
...,...,...,...,...,...
9952,Chinese Pavilion,"Bad rating is mainly because of "" Chicken Bone...","[('bad', 'NNP'), ('rating', 'NN'), ('mainly', ...","Chinese, Seafood",chicken bone
9952,Chinese Pavilion,"Bad rating is mainly because of "" Chicken Bone...","[('bad', 'NNP'), ('rating', 'NN'), ('mainly', ...","Chinese, Seafood",plum sauce
9952,Chinese Pavilion,"Bad rating is mainly because of "" Chicken Bone...","[('bad', 'NNP'), ('rating', 'NN'), ('mainly', ...","Chinese, Seafood",spicy sauce
9952,Chinese Pavilion,"Bad rating is mainly because of "" Chicken Bone...","[('bad', 'NNP'), ('rating', 'NN'), ('mainly', ...","Chinese, Seafood",fried rice veg hakka noodles


In [15]:
meals_ser = meals_df['meals']

In [16]:
meals_ser = meals_ser.str.lower()

In [17]:
meals_ser

2                penne alfredo pasta
4                    kodi drumsticks
4              basket mutton biryani
7                               corn
7                          tawa fish
                    ...             
9952                    chicken bone
9952                      plum sauce
9952                     spicy sauce
9952    fried rice veg hakka noodles
9954          pepper chicken starter
Name: meals, Length: 7197, dtype: object

In [18]:

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [19]:
lemmatizer = WordNetLemmatizer()

meals_lemma = meals_ser.apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(x)]))

In [20]:
meals_ser

2                penne alfredo pasta
4                    kodi drumsticks
4              basket mutton biryani
7                               corn
7                          tawa fish
                    ...             
9952                    chicken bone
9952                      plum sauce
9952                     spicy sauce
9952    fried rice veg hakka noodles
9954          pepper chicken starter
Name: meals, Length: 7197, dtype: object

In [21]:
meals_lemma

2               penne alfredo pasta
4                    kodi drumstick
4             basket mutton biryani
7                              corn
7                         tawa fish
                   ...             
9952                   chicken bone
9952                     plum sauce
9952                    spicy sauce
9952    fried rice veg hakka noodle
9954         pepper chicken starter
Name: meals, Length: 7197, dtype: object

In [22]:
diff = meals_ser.compare(meals_lemma)

In [23]:
diff

,self,other
4,kodi drumsticks,kodi drumstick
14,corn cheese balls,corn cheese ball
15,cheese balls,cheese ball
20,honey chilli lotus stems,honey chilli lotus stem
20,basket biryanis,basket biryani
...,...,...
9941,noodles brownie,noodle brownie
9942,black bean noodles,black bean noodle
9943,korean noodles,korean noodle
9950,fish cakes,fish cake


In [24]:
meals_lemma = meals_lemma.drop_duplicates()

IDEAS:

put into clusters like e.g. drink, dessert, main dish, soup (or more detailed)